In [0]:
# --- widgets (you can change these later) ---
dbutils.widgets.text("start_date", "2015-09-07")
dbutils.widgets.text("end_date",   "2015-09-08")
dbutils.widgets.text("api_key",    "DEMO_KEY")  # swap to your own if you have one

start_date = dbutils.widgets.get("start_date")
end_date   = dbutils.widgets.get("end_date")
api_key    = dbutils.widgets.get("api_key")

# --- call NASA NEO feed ---
import requests, json
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}"
r = requests.get(url, timeout=60)
r.raise_for_status()
payload = r.json()

# --- quick smoke checks ---
neo_map = payload.get("near_earth_objects", {})
num_days = len(neo_map)
num_neos  = sum(len(v) for v in neo_map.values())

print(f"Fetched {num_neos} NEOs across {num_days} day(s) for {start_date}..{end_date}")
# peek the first object to confirm structure
first_day = sorted(neo_map.keys())[0]
first_obj = neo_map[first_day][0]
print("Sample NEO:", first_obj.get("name"), "| hazardous:", first_obj.get("is_potentially_hazardous_asteroid"))
print("Sample approaches:", len(first_obj.get("close_approach_data", [])))


In [0]:
#Create a working database for the table
spark.sql("CREATE DATABASE IF NOT EXISTS nasa_neo_db")
spark.sql("USE nasa_neo_db")

In [0]:
#Ingest raw JSON → bronze_neo_feed table
#import json
#from pyspark.sql.functions import current_timestamp,lit

# Reuse variables from Step 1: start_date, end_date, api_key, payload, url
#bronze_df = spark.createDataFrame(
#    [(start_date, end_date, json.dumps(payload), url)],
#    ["start_date","end_date","raw_json","source_url"]
#).withColumn("ingested_at", current_timestamp())

#bronze_df.write.mode("append").format("delta").saveAsTable("bronze_neo_feed")
#print("Bronze write complete.")


Step 2 (CE-safe) — “Land” raw JSON as a TEMP VIEW

In [0]:
#Build a Bronze DataFrame and register a temp view

import json
from pyspark.sql.functions import current_timestamp

# Create a one-row DataFrame holding the raw payload
bronze_df = spark.createDataFrame(
    [(start_date, end_date, json.dumps(payload), url)],
    ["start_date","end_date","raw_json","source_url"]
).withColumn("ingested_at", current_timestamp())

# Register as a TEMP VIEW (session-scoped; no persistence needed for CE)
bronze_df.createOrReplaceTempView("bronze_neo_feed")

print("TEMP VIEW 'bronze_neo_feed' is ready (no files/tables written).")


In [0]:
# Quick validation
spark.sql("""
SELECT
  start_date, end_date,
  length(raw_json) AS raw_len,
  source_url,
  ingested_at
FROM bronze_neo_feed
ORDER BY ingested_at DESC
LIMIT 3
""").display()

In [0]:
spark.sql("""
SELECT start_date, end_date, LENGTH(raw_json) AS raw_len, source_url, ingested_at
FROM bronze_neo_feed
""").show(truncate=False)

###STEP -3
#####Normalize JSON to Silver tables: neo and close_approach
######When you normalize to Silver, read from the temp view instead of a table or DBFS path. Use the serverless-safe parsing (no RDDs) like this at the top of Step 3

In [0]:
from pyspark.sql import functions as F, types as T

# --- schemas (same as before) ---
cad_schema = T.StructType([
    T.StructField("close_approach_date", T.StringType()),
    T.StructField("epoch_date_close_approach", T.LongType()),
    T.StructField("relative_velocity", T.StructType([
        T.StructField("kilometers_per_second", T.StringType()),
    ])),
    T.StructField("miss_distance", T.StructType([
        T.StructField("kilometers", T.StringType()),
        T.StructField("astronomical", T.StringType()),
    ])),
    T.StructField("orbiting_body", T.StringType())
])

neo_obj_schema = T.StructType([
    T.StructField("neo_reference_id", T.StringType()),
    T.StructField("name", T.StringType()),
    T.StructField("absolute_magnitude_h", T.DoubleType()),
    T.StructField("estimated_diameter", T.StructType([
        T.StructField("kilometers", T.StructType([
            T.StructField("estimated_diameter_min", T.DoubleType()),
            T.StructField("estimated_diameter_max", T.DoubleType()),
        ]))
    ])),
    T.StructField("is_potentially_hazardous_asteroid", T.BooleanType()),
    T.StructField("nasa_jpl_url", T.StringType()),
    T.StructField("close_approach_data", T.ArrayType(cad_schema))
])

root_schema = T.StructType([
    T.StructField("near_earth_objects",
                  T.MapType(T.StringType(), T.ArrayType(neo_obj_schema)))
])

# --- read from the TEMP VIEW (no RDD, no files) ---
bronze_latest = spark.table("bronze_neo_feed")

parsed = bronze_latest.select(
    F.from_json(F.col("raw_json"), root_schema).alias("j")
)

dated = parsed.select(F.explode(F.col("j.near_earth_objects")).alias("approach_date", "neos"))
neos_flat = dated.select(
    F.to_date("approach_date").alias("approach_date"),
    F.explode("neos").alias("neo")
)

neo_df = (neos_flat
    .select(
        F.col("neo.neo_reference_id").alias("neo_id"),
        F.col("neo.name").alias("name"),
        F.col("neo.absolute_magnitude_h").alias("absolute_magnitude_h"),
        F.col("neo.estimated_diameter.kilometers.estimated_diameter_min").cast("double").alias("est_diam_min_km"),
        F.col("neo.estimated_diameter.kilometers.estimated_diameter_max").cast("double").alias("est_diam_max_km"),
        F.col("neo.is_potentially_hazardous_asteroid").alias("is_potentially_hazardous_asteroid"),
        F.col("neo.nasa_jpl_url").alias("nasa_jpl_url"),
    )
    .dropDuplicates(["neo_id"])
)

approach_df = (neos_flat
    .select(
        F.col("neo.neo_reference_id").alias("neo_id"),
        F.explode("neo.close_approach_data").alias("cad")
    )
    .select(
        F.col("neo_id"),
        F.to_date("cad.close_approach_date").alias("close_approach_date"),
        F.col("cad.epoch_date_close_approach").cast("long").alias("epoch_close_approach"),
        F.col("cad.relative_velocity.kilometers_per_second").cast("double").alias("relative_velocity_kms"),
        F.col("cad.miss_distance.kilometers").cast("double").alias("miss_distance_km"),
        F.col("cad.miss_distance.astronomical").cast("double").alias("miss_distance_au"),
        F.col("cad.orbiting_body").alias("orbiting_body"),
    )
)

print("NEO rows:", neo_df.count(), "| Approaches:", approach_df.count())


#Step 4
###Build the Gold layer as temp views and run the analysis

In [0]:
#4A) Register temp views for SQL
#register temp views for analysis
neo_df.createOrReplaceTempView("neo")
approach_df.createOrReplaceTempView("close_approach")

In [0]:
%sql
--4B) Create analysis-friendly Gold views (size bins, joins, distance bins)
--NEO profile with a handy midpoint diameter and size bins
CREATE OR REPLACE TEMP VIEW neo_profile AS
SELECT
  n.neo_id,
  n.name,
  n.absolute_magnitude_h,
  (n.est_diam_min_km + n.est_diam_max_km)/2.0 AS est_diam_mid_km,
  CASE
    WHEN (n.est_diam_min_km + n.est_diam_max_km)/2.0 < 0.1 THEN 'Tiny (<100m)'
    WHEN (n.est_diam_min_km + n.est_diam_max_km)/2.0 < 0.3 THEN 'Small (100–300m)'
    WHEN (n.est_diam_min_km + n.est_diam_max_km)/2.0 < 1.0 THEN 'Medium (300m–1km)'
    ELSE 'Large (≥1km)'
  END AS size_bin,
  n.is_potentially_hazardous_asteroid,
  n.nasa_jpl_url
FROM neo n;

-- One row per approach with hazard + size carried through
CREATE OR REPLACE TEMP VIEW approach_facts AS
SELECT
  a.neo_id,
  p.name,
  p.size_bin,
  p.is_potentially_hazardous_asteroid,
  a.close_approach_date,
  a.relative_velocity_kms,
  a.miss_distance_km,
  a.miss_distance_au,
  a.orbiting_body
FROM close_approach a
JOIN neo_profile p USING (neo_id);

-- Distance bins for Story #2
CREATE OR REPLACE TEMP VIEW approach_facts_binned AS
SELECT *,
  CASE
    WHEN miss_distance_au <= 0.05 THEN '≤0.05 AU'
    WHEN miss_distance_au <= 0.1  THEN '0.05–0.1 AU'
    ELSE '>0.1 AU'
  END AS distance_bin
FROM approach_facts;


##4C) Queries that answer the two user stories

###Story 1 — “Comprehensive analysis + hazard classification”

In [0]:
%sql
-- NEO profiles (sort by size)
SELECT neo_id, name, round(est_diam_mid_km,4) AS diam_km_mid,
       size_bin, is_potentially_hazardous_asteroid
FROM neo_profile
ORDER BY est_diam_mid_km DESC;

-- Closest passes first (adjust LIMIT for a quick peek)
SELECT neo_id, name, close_approach_date, orbiting_body,
       round(miss_distance_km,0) AS miss_km,
       round(relative_velocity_kms,2) AS rel_vel_kms,
       is_potentially_hazardous_asteroid
FROM approach_facts
ORDER BY miss_distance_km ASC
LIMIT 50;


Story 2 — “Frequency by size, distance, hazard”

In [0]:
%sql
-- Frequency over time
SELECT close_approach_date, COUNT(*) AS approaches
FROM approach_facts
GROUP BY close_approach_date
ORDER BY close_approach_date;

-- By size bin
SELECT size_bin, COUNT(*) AS approaches
FROM approach_facts
GROUP BY size_bin
ORDER BY approaches DESC;

-- By distance bin and hazard flag
SELECT distance_bin, is_potentially_hazardous_asteroid, COUNT(*) AS approaches
FROM approach_facts_binned
GROUP BY distance_bin, is_potentially_hazardous_asteroid
ORDER BY distance_bin, is_potentially_hazardous_asteroid DESC;


4D) (Optional) Parameterized date filtering for demos

In [0]:
dbutils.widgets.text("from_date", "2015-09-07")
dbutils.widgets.text("to_date",   "2015-09-08")
from_date = dbutils.widgets.get("from_date")
to_date   = dbutils.widgets.get("to_date")


In [0]:
spark.sql(f"""
SELECT *
FROM approach_facts
WHERE close_approach_date BETWEEN DATE('{from_date}') AND DATE('{to_date}')
ORDER BY miss_distance_km ASC
""").display()
